###  모델 프리징 (Model Freezing)

전이 학습 중 잘 학습 된 모델을 가져와 원하는 연구에 사용할 수 있다. 데이터가 유사한 경우에는 추가적으로 전체 학습 없이도 좋은 성능이 나올 수 있다. 따라서 피쳐 추출에 해당하는 합성곱 층의 변수를 업데이트 하지 않고 분류 파트에 해당하는 fully connected layer의 변수만 업데이트 할 수 있는데 이 때 변수가 없데이트 되지 않게 변수를 얼린다고 하여 이를 프리징(Freezing)이라고 한다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim
from tqdm import trange 

#### 1. GPU 연산 확인

In [5]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(f"{device} is available")

mps is available


#### 2. CIFAR10 데이터 불러오기

In [6]:
# 데이터 불러오기 및 전처리 작업
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

#### 3. Pretrained model 불러오기

In [8]:
# ResNet-18 불러오기
# weights='DEFAULT' : ResNet-18 구조를 쓰되, ImageNet으로 이미 똑똑해진 모델을 불러온다.
# weights=False를 하면 ResNet18 구조만 불러온다.
# 모델과 텐서에 .to(device)를 붙여야만 GPU 연산이 가능하다.

model = torchvision.models.resnet18(weights='DEFAULT')

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) # feature map의 크기가 유지된다.
model.fc = nn.Linear(512, 10)
model = model.to(device)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### 4. 모델 프리징

In [ ]:
# 파라메타 번호 확인 하기
# 각 층은 보통 2가지 파라미터를 가진다.
# 1. weight(가중치)
# 2. bias(편향, 옵션)
i = 0

for name, param in model.named_parameters(): # 모델안의 학습 가능한 가중치와 편향을 하나하나 가져온다. -> 결과를 보면 모두 각각의 층의 weight와 bias인 것을 알 수 있다.
  print(i,name)

  i+= 1

0 conv1.weight
1 bn1.weight
2 bn1.bias
3 layer1.0.conv1.weight
4 layer1.0.bn1.weight
5 layer1.0.bn1.bias
6 layer1.0.conv2.weight
7 layer1.0.bn2.weight
8 layer1.0.bn2.bias
9 layer1.1.conv1.weight
10 layer1.1.bn1.weight
11 layer1.1.bn1.bias
12 layer1.1.conv2.weight
13 layer1.1.bn2.weight
14 layer1.1.bn2.bias
15 layer2.0.conv1.weight
16 layer2.0.bn1.weight
17 layer2.0.bn1.bias
18 layer2.0.conv2.weight
19 layer2.0.bn2.weight
20 layer2.0.bn2.bias
21 layer2.0.downsample.0.weight
22 layer2.0.downsample.1.weight
23 layer2.0.downsample.1.bias
24 layer2.1.conv1.weight
25 layer2.1.bn1.weight
26 layer2.1.bn1.bias
27 layer2.1.conv2.weight
28 layer2.1.bn2.weight
29 layer2.1.bn2.bias
30 layer3.0.conv1.weight
31 layer3.0.bn1.weight
32 layer3.0.bn1.bias
33 layer3.0.conv2.weight
34 layer3.0.bn2.weight
35 layer3.0.bn2.bias
36 layer3.0.downsample.0.weight
37 layer3.0.downsample.1.weight
38 layer3.0.downsample.1.bias
39 layer3.1.conv1.weight
40 layer3.1.bn1.weight
41 layer3.1.bn1.bias
42 layer3.1.conv2.wei

In [13]:
# frozen이라는 범위를 만들어 (3부터 시작해서 3씩 증가하며 60 미만까지 숫자를 생성한다.)
# 해당 범위의 숫자는 특정 파라미터 번호를 의미하며, 이 번호에 해당하는 파라미터들은 학습하지 않고 고정된다.
frozen = range(3,60,3)

for i, (name, param) in enumerate(model.named_parameters()):
    
    if i in frozen:
        param.requires_grad = False # 학습할 때 업데이트 되지 않는다. ~ gradient가 계산되지 않는다.

In [14]:
# requires_grad 확인
print(model.layer4[1].conv2.weight.requires_grad)
print(model.fc.weight.requires_grad)

False
True


#### 5. 손실함수와 최적화 방법 정의

In [15]:
criterion = nn.CrossEntropyLoss() # 분류 문제에서 자주 쓰이는 손실 함수
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

#### 6. 손실함수와 최적화 방법 정의

In [ ]:
num_epochs = 20
ls = 2 # 어지간한 loss는 2보다 작다.
pbar = trange(num_epochs)

for epoch in pbar:
    correct = 0
    total = 0
    running_loss = 0.0
    for data in trainloader:
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.detach(), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    cost = running_loss / len(trainloader)   
    acc = 100*correct / total

    if cost < ls:
        ls = cost
        torch.save(model.state_dict(), './models/cifar10_resnet_frozen.pth')   

    pbar.set_postfix({'loss' : cost, 'train acc' : acc}) 


#### 7. 모델 평가

In [ ]:
model.load_state_dict(torch.load('./models/cifar10_resnet_frozen.pth'))

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model.eval()
    
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total)) 

# 모델 프리징 미적용시: 학습 27분 소요, 정확도 85%
# 모델 프리징 적용시: 학습 21분 소요, 정확도 82%
# 빠르지만 성능은 조금 손해 볼 수 있다는 결과가 나왔다.